In [1]:
import numpy as np
import open3d as o3d

# Step 1: Read the .bin file
def read_bin_file(bin_file_path):
    point_cloud = np.fromfile(bin_file_path, dtype=np.float32).reshape(-1, 4)
    return point_cloud[:, :3]  # Keep only X, Y, Z


# Step 2: Convert point cloud data to Open3D format
def convert_bin_to_ply(bin_file_path, output_ply_path):
    # Read point cloud data
    points = read_bin_file(bin_file_path)
    
    # Create an Open3D point cloud object
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)
    
    # Save the point cloud as a .ply file
    o3d.io.write_point_cloud(output_ply_path, point_cloud)
    print(f"Point cloud saved as {output_ply_path}")

# Step 3: Use the conversion function to convert .bin to .ply
bin_file_1 = "/project/bli4/maps/wacv/GeoTransformer/data/Kitti/sequences/08/velodyne/001786.bin"
bin_file_2 = "/project/bli4/maps/wacv/GeoTransformer/data/Kitti/sequences/08/velodyne/001777.bin"

# Convert to .ply files
convert_bin_to_ply(bin_file_1, "/project/bli4/maps/wacv/GeoTransformer/data/Rellis/kitti_001786.ply")
convert_bin_to_ply(bin_file_2, "/project/bli4/maps/wacv/GeoTransformer/data/Rellis/kitti_001777.ply")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Point cloud saved as /project/bli4/maps/wacv/GeoTransformer/data/Rellis/kitti_001786.ply
Point cloud saved as /project/bli4/maps/wacv/GeoTransformer/data/Rellis/kitti_001777.ply


In [4]:
import numpy as np
import open3d as o3d  
from sklearn.neighbors import NearestNeighbors  # Calculate alignment error

def load_point_cloud(ply_file):
    pcd = o3d.io.read_point_cloud(ply_file)
    points = np.asarray(pcd.points)
    return points

def apply_transform(points, transform):
    # Convert point cloud to homogeneous coordinates
    ones = np.ones((points.shape[0], 1))
    homogeneous_points = np.hstack((points, ones))
    # Application of transformation matrix
    transformed_points = (transform @ homogeneous_points.T).T
    # Return non-homogeneous coordinates
    return transformed_points[:, :3]

def compute_alignment_error(source_points, target_points):
    nbrs = NearestNeighbors(n_neighbors=1).fit(target_points)
    distances, _ = nbrs.kneighbors(source_points)
    return np.mean(distances)

# Point cloud file path(kitti)
p1_file = "/project/bli4/maps/wacv/GeoTransformer/data/Rellis/kitti_001786.ply"
p2_file = "/project/bli4/maps/wacv/GeoTransformer/data/Rellis/kitti_001777.ply"

# # Point cloud file path(rellis)
# p1_file = "/project/bli4/autoai/joyang/GeoTransformer/data/Rellis/sequences/00004/os1_cloud_node_color_ply/000040.ply"
# p2_file = "/project/bli4/autoai/joyang/GeoTransformer/data/Rellis/sequences/00004/os1_cloud_node_color_ply/000240.ply"

# Load point cloud data
p1_points = load_point_cloud(p1_file)
p2_points = load_point_cloud(p2_file)

# Transformation matrix from kitti gt
transform = np.array([[ 9.99981321e-01,  3.78943044e-03, -4.80179457e-03,
        -9.68984093e+00],
       [-3.80175465e-03,  9.99989547e-01, -2.56015328e-03,
        -5.39332090e-02],
       [ 4.79204138e-03,  2.57836601e-03,  9.99985241e-01,
        -2.01965901e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

# # Transformation matrix from rellis gt
# transform = np.array([[ 0.99812302, -0.04881774, -0.03697775,  7.86450395],
#        [ 0.04808394,  0.99863291, -0.02049177,  1.08264397],
#        [ 0.03792745,  0.0186753 ,  0.99910529,  0.12559312],
#        [ 0.        ,  0.        ,  0.        ,  1.        ]])

# Scenario 1: Assume that transform is the transformation from p1 to p2.
p1_transformed = apply_transform(p1_points, transform)

# Scenario 2: Assume that transform is the transformation from p2 to p1.
# Calculate the inverse matrix of transform.
transform_inv = np.linalg.inv(transform)
p2_transformed = apply_transform(p2_points, transform_inv)

# Calculate alignment error
error_p1_to_p2 = compute_alignment_error(p1_transformed, p2_points)
error_p2_to_p1 = compute_alignment_error(p2_transformed, p1_points)

print(f"Error（p1 to p2）：{error_p1_to_p2}")
print(f"Error（p2 to p1）：{error_p2_to_p1}")

if error_p1_to_p2 < error_p2_to_p1:
    print("transform is the transformation matrix from p1 to p2")
else:
    print("transform is the transformation matrix from p2 to p1")


Error（p1 to p2）：1.1436139778232353
Error（p2 to p1）：0.8922462687408756
transform is the transformation matrix from p2 to p1
